In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

from sklearn.cross_validation import *
from sklearn.grid_search import GridSearchCV

train = pd.read_csv("F:\\study\\ml\\DataSet\\Hbc_csv\\hbcTrain.csv")
test = pd.read_csv("F:\\study\\ml\\DataSet\\Hbc_csv\\hbcTest.csv")

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
#train.head()
features=list(train.columns)
features.remove('actionType')
print(features)

train = train.fillna(-999)
test = test.fillna(-999) 

['userid', 'actionTime', 'Horderid', 'orderTime', 'orderType', 'cityNo', 'countryNo', 'continentNo', 'Corderid', 'rating', 'genderNo', 'province', 'provinceNo', 'age', 'ageNo', 'PorderType']


In [4]:
xgb_model=xgb.XGBClassifier()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['binary:logistic'],
              'learning_rate': [0.1], #so called `eta` value
              'max_depth': [6],
              'min_child_weight': [11],
              'silent': [0],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [5], #number of trees, change it to 1000 for better results
              'missing':[-999]}

clf = GridSearchCV(xgb_model, parameters, n_jobs=5,
                   cv=StratifiedKFold(train['actionType'], n_folds=5, shuffle=True), 
                   scoring='roc_auc',
                   verbose=2, refit=True)

clf.fit(train[features], train["actionType"])

best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])
print('Raw AUC score:', score)
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))
    


TypeError: __init__() got an unexpected keyword argument 'num_round'

In [14]:
train.head()

,userid,actionType,actionTime,Horderid,orderTime,orderType,cityNo,countryNo,continentNo,Corderid,rating,genderNo,province,provinceNo,age,ageNo,PorderType
0,100000000013,0,100000000013,1,1474300753,1.000000e+11,1000015.0,1.481715e+09,108.0,27.0,4.0,1.000000e+11,1000015.0,100000000013,1.0,5.0,-999.0
1,100000000013,0,100000000013,1,1474300753,1.000000e+11,1000014.0,1.501960e+09,26.0,2.0,1.0,1.000000e+11,1000015.0,100000000013,1.0,5.0,-999.0
2,100000000013,0,100000000013,5,1474300763,1.000000e+11,1000015.0,1.481715e+09,108.0,27.0,4.0,1.000000e+11,1000015.0,100000000013,1.0,5.0,-999.0
3,100000000013,0,100000000013,5,1474300763,1.000000e+11,1000014.0,1.501960e+09,26.0,2.0,1.0,1.000000e+11,1000015.0,100000000013,1.0,5.0,-999.0
4,100000000013,0,100000000013,6,1474300874,1.000000e+11,1000015.0,1.481715e+09,108.0,27.0,4.0,1.000000e+11,1000015.0,100000000013,1.0,5.0,-999.0


In [5]:

train_label=train['actionType']
train_data=train.drop('actionType',axis=1)
x_train,x_val,y_train,y_val=train_test_split(train_data,train_label,test_size=0.2,random_state=100)
x_train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2110520 entries, 111123 to 1873416
Data columns (total 16 columns):
userid         int64
actionTime     int64
Horderid       int64
orderTime      int64
orderType      float64
cityNo         float64
countryNo      float64
continentNo    float64
Corderid       float64
rating         float64
genderNo       float64
province       float64
provinceNo     int64
age            float64
ageNo          float64
PorderType     float64
dtypes: float64(11), int64(5)
memory usage: 273.7 MB


In [6]:
param = {'learning_rate' : 0.1,
        'n_estimators': 1000,
        'max_depth': 4,
        'min_child_weight': 11,
        'gamma': 0,
        'subsample': 0.7,
        'colsample_bytree': 0.75,
        'eta': 0.05,
        'silent': 1,
        'objective':
        'binary:logistic',
        'scale_pos_weight':1,
        'missing':[-999]}
num_round =500
param['eval_metric'] = 'auc'

dtrain = xgb.DMatrix(x_train,label=y_train)
dtest = dtrain
evallist  = [(dtest,'eval'), (dtrain,'train')]
bst=xgb.train(param,dtrain,num_round,evallist,early_stopping_rounds=100)



#bst = xgb.train( param, dtrain, num_round, evallist )
bst.dump_model('F:\\study\\ml\\DataSet\\Hbc_csv\\dump.raw.txt')

[0]	eval-auc:0.760585	train-auc:0.760585
Multiple eval metrics have been passed: 'train-auc' will be used for early stopping.

Will train until train-auc hasn't improved in 100 rounds.
[1]	eval-auc:0.766537	train-auc:0.766537
[2]	eval-auc:0.772117	train-auc:0.772117
[3]	eval-auc:0.777335	train-auc:0.777335
[4]	eval-auc:0.782834	train-auc:0.782834
[5]	eval-auc:0.789654	train-auc:0.789654
[6]	eval-auc:0.792086	train-auc:0.792086
[7]	eval-auc:0.793135	train-auc:0.793135
[8]	eval-auc:0.795067	train-auc:0.795067
[9]	eval-auc:0.796122	train-auc:0.796122
[10]	eval-auc:0.801215	train-auc:0.801215
[11]	eval-auc:0.807208	train-auc:0.807208
[12]	eval-auc:0.81073	train-auc:0.81073
[13]	eval-auc:0.811871	train-auc:0.811871
[14]	eval-auc:0.812218	train-auc:0.812218
[15]	eval-auc:0.815804	train-auc:0.815804
[16]	eval-auc:0.81626	train-auc:0.81626
[17]	eval-auc:0.818191	train-auc:0.818191
[18]	eval-auc:0.818493	train-auc:0.818493
[19]	eval-auc:0.82044	train-auc:0.82044
[20]	eval-auc:0.821632	train-auc

[191]	eval-auc:0.90384	train-auc:0.90384
[192]	eval-auc:0.90404	train-auc:0.90404
[193]	eval-auc:0.904307	train-auc:0.904307
[194]	eval-auc:0.904461	train-auc:0.904461
[195]	eval-auc:0.904848	train-auc:0.904848
[196]	eval-auc:0.905188	train-auc:0.905188
[197]	eval-auc:0.905255	train-auc:0.905255
[198]	eval-auc:0.905352	train-auc:0.905352
[199]	eval-auc:0.905626	train-auc:0.905626
[200]	eval-auc:0.905693	train-auc:0.905693
[201]	eval-auc:0.905866	train-auc:0.905866
[202]	eval-auc:0.90589	train-auc:0.90589
[203]	eval-auc:0.906051	train-auc:0.906051
[204]	eval-auc:0.906247	train-auc:0.906247
[205]	eval-auc:0.906363	train-auc:0.906363
[206]	eval-auc:0.906538	train-auc:0.906538
[207]	eval-auc:0.90683	train-auc:0.90683
[208]	eval-auc:0.906971	train-auc:0.906971
[209]	eval-auc:0.907012	train-auc:0.907012
[210]	eval-auc:0.907177	train-auc:0.907177
[211]	eval-auc:0.907389	train-auc:0.907389
[212]	eval-auc:0.907576	train-auc:0.907576
[213]	eval-auc:0.907971	train-auc:0.907971
[214]	eval-auc:0.90

[383]	eval-auc:0.927346	train-auc:0.927346
[384]	eval-auc:0.927359	train-auc:0.927359
[385]	eval-auc:0.927487	train-auc:0.927487
[386]	eval-auc:0.927574	train-auc:0.927574
[387]	eval-auc:0.927619	train-auc:0.927619
[388]	eval-auc:0.927777	train-auc:0.927777
[389]	eval-auc:0.92792	train-auc:0.92792
[390]	eval-auc:0.928014	train-auc:0.928014
[391]	eval-auc:0.928069	train-auc:0.928069
[392]	eval-auc:0.928166	train-auc:0.928166
[393]	eval-auc:0.928353	train-auc:0.928353
[394]	eval-auc:0.928438	train-auc:0.928438
[395]	eval-auc:0.928581	train-auc:0.928581
[396]	eval-auc:0.928703	train-auc:0.928703
[397]	eval-auc:0.928783	train-auc:0.928783
[398]	eval-auc:0.928824	train-auc:0.928824
[399]	eval-auc:0.928834	train-auc:0.928834
[400]	eval-auc:0.928888	train-auc:0.928888
[401]	eval-auc:0.928933	train-auc:0.928933
[402]	eval-auc:0.928947	train-auc:0.928947
[403]	eval-auc:0.929089	train-auc:0.929089
[404]	eval-auc:0.929142	train-auc:0.929142
[405]	eval-auc:0.929211	train-auc:0.929211
[406]	eval-au